In [ ]:
#Import the necessary libraries
import cv2  
import os
import pandas as pd
from scipy import stats

In [ ]:
#loading the pre-trained deep learning model trained on the Microsoft COCO dataset

config_file="ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
frozen_model="frozen_inference_graph.pb"
model = cv2.dnn_DetectionModel(frozen_model, config_file)

classLabels = [] #empty list of python 
file_name = "labels.txt"
with open(file_name, "rt") as fpt: 
    classLabels = fpt.read().rstrip("\n").split("\n")

#References
#https://www.youtube.com/watch?v=RFqvTmEFtOE

In [ ]:
#objects which the deep learning model is trained to detect
print(classLabels)

In [ ]:
#setting up the pre-trained model
model.setInputSize(320,320)
model.setInputScale(1.0/127.5) #255/2 = 127.5 (255 mostly grey level)
model.setInputMean((127.5,127.5,127.5)) #taking the mean, mobilenet => takes [-1,1] 
model.setInputSwapRB(True)

In [ ]:
#for the font and font size of the bounding boxes for objects detected in images
font_scale = 1
font = cv2.FONT_HERSHEY_PLAIN 

In [ ]:
#Function loading the MRT images into the object detection model and retrieving the results
def load_images_from_folder(folder):
    default_value = 0
    obj_bin = dict.fromkeys(classLabels,default_value)
    obj_data=[]
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        #draw bounding box on the objects detected by the model in the image
        #if the confidence threshold is at least 50%
        ClassIndex, confidence, bbox = model.detect(img, confThreshold=0.50)
        #detect objects in each image and taking note of the counts
        for obj_identified in ClassIndex:
            #obj_identified-1 instead of obj_indentified as the indexes are read in this manner
            idx=classLabels[obj_identified-1]
            obj_bin[idx]+=1
        obj_data.append(obj_bin)
        obj_bin=obj_bin.fromkeys(obj_bin, 0)
    #export the result (object detection histogram) to excel sheet
    df=pd.DataFrame.from_dict(obj_data, orient='columns')
    with pd.ExcelWriter('objects_detection2.xlsx',mode='wb') as writer:  
        df.to_excel(writer, sheet_name='obj_hist')

#References
#https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
#https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe

In [ ]:
#calling the function
load_images_from_folder("Survey_images/sg_mrt_stations")

In [ ]:
# peform correlation and saving the results

#reading the data (from excel ) using pandas
df=pd.read_excel("objects_detection.xlsx",sheet_name="Updated4corr",header=[0],index_col=[0], usecols="B:M")
#using pearson's correlation
df2=df.corr(method ='pearson')
#export the results to excel sheet
with pd.ExcelWriter('objects_detection.xlsx',engine = 'openpyxl',mode='a') as writer:  
    df2.to_excel(writer, sheet_name='obj_corr')

#References
#https://stackoverflow.com/questions/42370977/how-to-save-a-new-sheet-in-an-existing-excel-file-using-pandas

In [ ]:
# retrieve p-values of the correlation results 

#reading the data (from excel ) using pandas
df=pd.read_excel("objects_detection.xlsx",sheet_name="Updated4corr",header=[0],index_col=[0], usecols="B:M")

#calculating the p-values of the correlation results
def pearsonr_pval(x,y):
    return stats.pearsonr(x,y)[1]

corr_p = df.corr(method=pearsonr_pval)

#export the p-values of the correlation results to excel sheet
with pd.ExcelWriter('objects_detection.xlsx',engine = 'openpyxl',mode='a') as writer:  
    corr_p.to_excel(writer, sheet_name='obj_corr_p')

#References
#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr
#https://stackoverflow.com/questions/52741236/how-to-calculate-p-values-for-pairwise-correlation-of-columns-in-pandas
#https://stackoverflow.com/questions/42370977/how-to-save-a-new-sheet-in-an-existing-excel-file-using-pandas